### Dynamic Systems

Potentials x, model matrix A, voltage source b,,  we have
$$A^{T}C(b-Ax) = f $$
b is zero, since there is not potential source (aka volatage in electrical), we have
$$A^TCAx = f $$
say $A^TCA = K$, this is the Kirchkoff current term, then
$$Kx = f$$

### Network Problem

### Steady State Conduction

Transient conduction is essentially a network problem in dynamic systems.
If we have a 1D network with 4 nodes and 3 edges, for the incidence matrix $A$ the rows are the nodes and the columns are the edges. The arrow enters the node is 1 and leaves the node is -1.

In [2]:
import numpy as np

A = np.matrix('-1 0 0;1 -1 0;0 1 -1;0 0 1').T
print(A)

[[-1  1  0  0]
 [ 0 -1  1  0]
 [ 0  0 -1  1]]


In [3]:
u = np.array([10,20,30,40]).T
print(u)
dif = np.dot(A, u)
print(dif)

[10 20 30 40]
[[10 10 10]]


In [4]:
C = np.eye(3)*np.array([1,2,3])
print(C)

[[ 1.  0.  0.]
 [ 0.  2.  0.]
 [ 0.  0.  3.]]


In [5]:
K = np.dot(np.dot(A.T,C),A)
print(K)

[[ 1. -1.  0.  0.]
 [-1.  3. -2.  0.]
 [ 0. -2.  5. -3.]
 [ 0.  0. -3.  3.]]


In [6]:
x = np.array([10,10,10,40]).T
print()

In [9]:
ud = np.insert(np.diag(K,1),0,0)
d = np.diag(K)
ld = np.insert(np.diag(K,-1), len(d)-1,0)
ab = np.matrix([ud,d,ld])
print(ab)

[[ 0. -1. -2. -3.]
 [ 1.  3.  5.  3.]
 [-1. -2. -3.  0.]]


In [10]:
D = np.zeros(len(x))
D[0] = x[-1] - x[0]
print(D)

[ 30.   0.   0.   0.]


In [14]:
import scipy.linalg as linalg
Result = linalg.solve_banded((1,1),ab,D)

LinAlgError: singular matrix

In [5]:
# By Ben Kenney - http://www.benk.ca
# 1D Time dependent spherical diffusion equation
# dC/dt = K div(grad(C)) + 2K/x grad(c) on x=[0,1]
# BC: @x=0 C=1, @x=1 dC/dC=-1
# Time discretization using Crank Nicolson scheme
# Isn't python beautiful?

import scipy
import scipy.sparse as sparse
import scipy.sparse.linalg
import numpy

N = 50
dx = 1/(N-1.0)
delta = -1.0
K = 1.0

#grid points
x = numpy.linspace(0,1,N)

#create time steps
k = 0.5/100
TFinal = 1
NumOfTimeSteps = int(TFinal/k)

#initial solution
u = numpy.transpose([numpy.ones(N)*1.0])

#source term
F = numpy.transpose([numpy.zeros(N)])
F[-1]=2.0*K*delta*(1.0/dx+1.0)
print(F)

#create matrices with boundary conditions
A1=numpy.zeros([N])
A2=numpy.zeros([N])
A1[0]=0.0 # constant value boundary
for i in range(1,N-1):
        array = numpy.zeros([N])
        array[i-1:i-1+3] = [1,-2,1]
        A1=numpy.vstack([A1,array])
        array = numpy.zeros([N])
        #array[i-1:i-1+3] = [-1.0/x[i],0.0,1.0/x[i]] #x is the grid spacing
        array[i-1:i-1+3] = [-1.0,0.0,1.0]
        A2=numpy.vstack([A2,array])
array = numpy.zeros([N])
array[-2:]=[2,-2] #gradient boundary condition
A1=numpy.vstack([A1,array])
print(A1)
A1=A1*K/dx/dx
A1=scipy.sparse.csr_matrix(A1)
array = numpy.zeros([N])
A2=numpy.vstack([A2,array])
print(A2)
A2=A2*2.0*K/2.0/dx #note: grid factor, 1/x, is built into A2 matrix already
A2=scipy.sparse.csr_matrix(A2)

data = []

#identity matrix
I = sparse.identity(N)

print("Time step = %g \t Time = %g"%(0, 0))
print(u)
for i in range(NumOfTimeSteps):
        A = (I - k/2.0*A1 - k/2.0*A2)
        b = (I + k/2.0*A1 + k/2.0*A2)*u+k*F
        u = numpy.transpose(numpy.mat(sparse.linalg.spsolve(A, b)))
        print("Time step = %g \t Time = %g"%(i+1, k*(i+1)))
        data.append(u)
print(u[:,-1])

[[   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [   0.]
 [-100.]]
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 1. -2.  1. ...,  0.  0.  0.]
 [ 0.  1. -2. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ..., -2.  1.  0.]
 [ 0.  0.  0. ...,  1. -2.  1.]
 [ 0.  0.  0. ...,  0.  2. -2.]]
[[ 0.  0.  0. ...,  0.  0.  0.]
 [-1.  0.  1. ...,  0.  0.  0.]
 [ 0. -1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ..., -1.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
Time step = 0 	 Time = 0
[[ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.